### Pós-graduação em Ciência de Dados e Machine Learning

#### Disciplina: **Introdução a Redes Neurais**

#### Projeto Final para disciplina Introdução a Redes Neurais

<BR>
    

#### Nome dos integrantes:  
#### RA: Alana Paula Barbosa Mota


# ** RESUMO: Deep Transfer Learning for Image Classification - https://arxiv.org/abs/1811.02685**


O autor discute como, em muitas aplicações de aprendizado profundo, a disponibilidade de grandes volumes de dados rotulados é limitada, o que dificulta o treinamento de modelos complexos como redes neurais convolucionais (CNNs) do zero. Para lidar com isso, o transfer learning é uma solução eficaz. Ele propõe reutilizar modelos treinados em grandes conjuntos de dados, como o ImageNet, e adaptá-los para novas tarefas com menos dados.

**Técnicas e Teorias**



*   Transferência de aprendizado com CNNs: Utilização de ResNet como extrator de características.
*   Fine-Tuning: Ajustar camadas finais do modelo para novos dados, evitando overfitting.
*   Aumento de Dados: Geração de variações nas imagens de treino para maior robustez.
*   Uso de Redes Profundas Pré-Treinadas: O autor discute várias arquiteturas de redes convolucionais profundas (CNNs), como ResNet e VGG, que foram treinadas em conjuntos de dados extensos.

O artigo demonstra a eficácia do aprendizado por transferência para melhorar a precisão de classificação em tarefas com dados limitados e oferece insights práticos sobre como aplicar essas técnicas em projetos reais de deep learning. O uso de redes como ResNet50 possibilita soluções eficazes para a classificação de imagens, reduzindo a necessidade de grandes volumes de dados e alto custo computacional.


---
# Base de dados [Food-101 ](https://www.kaggle.com/datasets/hari31416/food-101?phase=FinishSSORegistration&returnUrl=%2Fdatasets%2Fhari31416%2Ffood-101%2Fversions%2F1%3Fresource%3Ddownload&SSORegistrationToken=CfDJ8KWOACvMaNFPiIJ818QpJK1Y5nCdELIm1VzSY_Wxic54iB_oX07r2BH8OqWSv15wKQD_w0AoE00A9yndUAA9YcQHz7Z1aGxz8uQ1ORHcM2FaIo4IEcmVCez-5Bpm30H6_rI8uVQ31pJhhaoUtEbfETvn8JGSrngXHznRaRTntqtNosNERtRu4hI36NkyYn6X1J0B9pK4cfsBwjMYJJbECXP2ZfIbeXz3NL77RJB56Em3xT1GxY3ev2EONNCNsCdyPZ-XsV7yQFLG-be5EArkHH380v6b0QqPDxmB7RYdFywUWpet92ojuZF0WDKl9_Jae2ki2GyV3-p-XoJrLPzJBbeBWSGtFw&DisplayName=Alana+Paula)


A base de dados Food-101 foi selecionada para ser utilizada no processo de identificação de alimentos. Essa abordagem tem se tornado amplamente utilizada no campo da nutrição, visando identificar diferentes tipos de alimentos por meio de imagens. Isso facilita cálculos nutricionais automáticos, avaliando características dos alimentos como macronutrientes, valor e qualidade nutricional. Além disso, essas técnicas vêm sendo aplicadas no desenvolvimento de dietas preventivas, principalmente voltadas ao combate da obesidade e outras doenças relacionadas à alimentação. Com a ajuda de redes neurais e algoritmos de deep learning, é possível criar sistemas que auxiliam na avaliação nutricional e no planejamento de dietas personalizadas.

**Aplicação para o Food-101:**


*   Uso de Modelos Pré-treinados: contém 101 classes de alimentos com 101.000 imagens. Modelos pré-treinados, como ResNet ou VGG, já foram treinados em grandes bases de dados como ImageNet, que inclui várias imagens relacionadas a alimentos.
*   Fine-Tuning para Identificação Específica: Ajustar as últimas camadas da rede (fine-tuning) permitirá que o modelo se especialize na distinção entre diferentes alimentos.
*   Aumento de Dados (Data Augmentation): Aumentar os dados da Food-101 por meio de técnicas como rotação, corte e flip horizontal pode gerar novas imagens sintéticas que ajudam a melhorar a generalização do modelo, como recomendado no artigo. Evitando o overfitting
*   Automação para Cálculos Nutricionais e Dieta: Ao identificar alimentos, o sistema pode ser integrado a um banco de dados nutricional, para calcular automaticamente valores nutricionais com base na imagem identificada.



In [ ]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pandas

In [ ]:
# Etapa 1: Importar as bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Caminho para o arquivo zip
dataset = '/content/drive/MyDrive/CNN/archive.zip'

# Criar um diretório para extrair os dados, se ainda não existir
extract_dir = '/content/drive/MyDrive/datasets/food-101'
os.makedirs(extract_dir, exist_ok=True)

# Extrair o arquivo zip
with zipfile.ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# Etapa 2: Carregar o modelo pré-treinado ResNet50 sem as camadas de saída (sem incluir a parte de classificação)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Etapa 3: Congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Etapa 4: Adicionar novas camadas de classificação no topo do modelo
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(101, activation='softmax')(x)  # 101 classes

In [ ]:
# Etapa 5: Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Etapa 6: Compilar o modelo com o otimizador Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Etapa 7: Pré-processar os dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Etapa 8: Carregar os dados de treino e validação
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/food-101/food-101/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/food-101/food-101/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Etapa 9: Configurar callbacks
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
# Etapa 10: Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=30,
    callbacks=[checkpoint, lr_scheduler]
)

In [ ]:
# Etapa 11: Avaliar o modelo
score = model.evaluate(validation_generator)
print(f'Acurácia no conjunto de validação: {score[1] * 100:.2f}%')

In [ ]:
# Etapa 12: Obter previsões e relatórios
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes))

In [ ]:
# Etapa 13: Visualizar gráficos de desempenho
plt.figure(figsize=(12, 4))

# Gráfico de Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Acurácia do Modelo')
plt.ylabel('Acurácia')
plt.xlabel('Épocas')
plt.legend()

In [ ]:
# Gráfico de Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Perda do Modelo')
plt.ylabel('Perda')
plt.xlabel('Épocas')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Etapa 10: Salvando o modelo treinado
model.save('modelo_transfer_nutri.h5')